In [2]:
%pip install backtrader awswrangler Pandas-TA 

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 410 kB 27.5 MB/s eta 0:00:01
     |████████████████████████████████| 238 kB 86.7 MB/s eta 0:00:01
     |████████████████████████████████| 115 kB 85.8 MB/s eta 0:00:01
     |████████████████████████████████| 207 kB 78.0 MB/s eta 0:00:01
     |████████████████████████████████| 26.7 MB 82.4 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 5.3 MB/s  eta 0:00:01
     |████████████████████████████████| 8.7 MB 81.3 MB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 99.5 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 87.4 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 21.7 MB/s eta 0:00:01
     |████████████████████████████████| 242 kB 87.8 MB/s eta 0:00:01
     |████████████████████████████████| 109 kB 115.4 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 18.8 MB/s eta 0:00:01
     |███████████████████

In [86]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pyspark

VBox()

An error was encountered:
Invalid status code '404' from http://localhost:8998/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [36]:
tcols = ['openbartime', 
         'firsttradetime',
         'highbidtime', 
         'highasktime', 
         'hightradetime',
         'lowbidtime', 
         'lowasktime', 
         'lowtradetime',
         'closebartime', 
         'lasttradetime']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
keep = ['firsttradeprice', 
        'hightradeprice', 
        'lowtradeprice', 
        'lasttradeprice', 
        'minspread', 
        'maxspread',
        'volumeweightprice', 
        'nbboquotecount', 
        'tradeatbid', 
        'tradeatbidmid', 
        'tradeatmid', 
        'tradeatmidask', 
        'tradeatask', 
        'volume', 
        'totaltrades', 
        'finravolume', 
        'finravolumeweightprice', 
        'uptickvolume', 
        'downtickvolume', 
        'repeatuptickvolume', 
        'repeatdowntickvolume', 
        'tradetomidvolweight', 
        'tradetomidvolweightrelative']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
trades =[
    'firsttradeprice', 
    'hightradeprice', 
    'lowtradeprice', 
    'lasttradeprice'
]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
df = spark.read.load('s3://algoseekraw/us-equity-1min-taq/**/**.csv.gz',format="csv",inferSchema="true",header="true")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+------------+------------------+------------+-----------+------------+-----------+------------------+---------------+--------------+------------------+------------+-----------+------------------+------------+-----------+------------------+--------------+-------------+------------------+-----------+----------+------------------+-----------+----------+------------------+-------------+------------+------------------+-------------+------------+-------------+------------+------------------+--------------+-------------+---------+---------+----------+-----------------+--------------+----------+-------------+----------+-------------+----------+--------------------+------+-----------+-----------+----------------------+------------+--------------+------------------+--------------------+-----------------+-------------------+---------------------------+-------------+-------------+
|    Date|Ticker|TimeBarStart|       OpenBarTime|OpenBidPrice|OpenBidSize|OpenAskPrice|OpenAskSize|  

In [13]:
df.select(countDistinct("Ticker")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+
|count(DISTINCT Ticker)|
+----------------------+
|                   100|
+----------------------+

In [70]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'NoneType' object has no attribute 'printSchema'
Traceback (most recent call last):
AttributeError: 'NoneType' object has no attribute 'printSchema'



In [76]:
df = df.withColumn('Date',to_date(df.Date.cast("string"),'yyyyMMdd'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
df = df.withColumn('datetime',to_timestamp(concat_ws(' ',df.Date,df.TimeBarStart)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df.withColumn('TimeBarStart',to_timestamp(df.TimeBarStart,'HH:mm')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------------------+------------------+------------+-----------+------------+-----------+------------------+---------------+--------------+------------------+------------+-----------+------------------+------------+-----------+------------------+--------------+-------------+------------------+-----------+----------+------------------+-----------+----------+------------------+-------------+------------+------------------+-------------+------------+-------------+------------+------------------+--------------+-------------+---------+---------+----------+-----------------+--------------+----------+-------------+----------+-------------+----------+--------------------+------+-----------+-----------+----------------------+------------+--------------+------------------+--------------------+-----------------+-------------------+---------------------------+-------------+-------------+
|      Date|Ticker|       TimeBarStart|       OpenBarTime|OpenBidPrice|OpenBidSize|OpenAskPr

In [62]:
df = df.withColumn('Minute',minute(df.datetime))
# df = df.withColumn('Minute',minute(df.datetime))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df.withColumn('OpenBarTime',to_timestamp(df.OpenBarTime)).show()


In [67]:
ticwindowSpec  = Window.partitionBy("Ticker").orderBy("datetime").rowsBetween(-1,0)
retWindowSpec  = Window.partitionBy("Ticker").orderBy("datetime").rowsBetween(-1,0)
rngTst = Window.partitionBy("Ticker").orderBy("datetime").rangeBetween(Window.unboundedPreceding,0)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df.withColumn('1minsma', avg(df.LastTradePrice).over(ticwindowSpec)).show()

In [68]:
winTst = Window.partitionBy("Ticker").orderBy("datetime")
df = df.withColumn('returns1min', (df['LastTradePrice'] / lag(df["LastTradePrice"],1).over(winTst))-1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+------------+------------------+------------+-----------+------------+-----------+--------------+---------------+--------------+------------------+------------+-----------+------------------+------------+-----------+-------------+--------------+-------------+------------------+-----------+----------+------------------+-----------+----------+------------+-------------+------------+------------------+-------------+------------+-------------+------------+-------------+--------------+-------------+---------+---------+----------+-----------------+--------------+----------+-------------+----------+-------------+----------+--------------------+------+-----------+-----------+----------------------+------------+--------------+------------------+--------------------+-----------------+-------------------+---------------------------+-------------+-------------+-------------------+------+-----------+
|      Date|Ticker|TimeBarStart|       OpenBarTime|OpenBidPrice|OpenBidSize|Open

In [ ]:
df = df.withColumn('returns2min', (df['LastTradePrice'] / lag(df["LastTradePrice"],2).over(winTst))-1)
df = df.withColumn('returns3min', (df['LastTradePrice'] / lag(df["LastTradePrice"],3).over(winTst))-1)
df = df.withColumn('returns4min', (df['LastTradePrice'] / lag(df["LastTradePrice"],4).over(winTst))-1)
df = df.withColumn('returns5min', (df['LastTradePrice'] / lag(df["LastTradePrice"],5).over(winTst))-1)

In [ ]:
by_ticker = df.groupBy("Ticker")
by_ticker_date = df.groupBy("Ticker","Date")

# Feature Engineering and Preprocessing

# SMA

In [ ]:
smaWinSpec  = Window.partitionBy("Ticker").orderBy("datetime").rowsBetween(-1,0)


In [ ]:
df.withColumn('1minsma', avg(df.LastTradePrice).over(ticwindowSpec)).show()

# EMA

# MACD

# Save partitioned table for persistence

In [79]:
df.write.partitionBy('Ticker','Date').parquet("s3://strategymodeldata/equityprice/emrout/equity_1min_taq.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
iid.write.partitionBy('symbol').parquet("s3://strategymodeldata/indicators/emrout/iid.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o520.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:195)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:194)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:232)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:229)
	

In [83]:
from tqdm import tqdm

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
df=spark.read.parquet("s3://strategymodeldata/equityprice/emrout/equity_1min_taq.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…